In [271]:
import time
import pandas as pd
import psycopg2
import datetime
from tqdm import trange



In [272]:
import matplotlib.pyplot as plt 
import matplotlib as mpl 
# %matplotlib inline 
import platform
from matplotlib import font_manager, rc # plt.rcParams['axes.unicode_minus'] = False 
if platform.system() == 'Darwin': rc('font', family='AppleGothic') 
elif platform.system() == 'Windows': 
    path = "c:/Windows/Fonts/malgun.ttf" 
    font_name = font_manager.FontProperties(fname=path).get_name() 
    rc('font', family=font_name) 
else: 
    print('Unknown system... sorry~~~~')

    # row 생략 없이 출력
pd.set_option('display.max_rows', 1000)
# col 생략 없이 출력
pd.set_option('display.max_columns', 1000)
plt.rcParams["figure.figsize"] = (14,4)
plt.rcParams['lines.linewidth'] = 2

In [273]:
# DB 연결 부분
uswer = 'dt.intern'
host = '10.80.0.234'
port = '5439'
dbname = 'fnf'
con = psycopg2.connect(dbname='fnf',host='10.80.0.234', port='5439', user = 'dt.intern', password='DTintn0926')
cur = con.cursor()


In [274]:
start = time.time()

total_shop_stock_ori = '''
SELECT vfs.weekkey,vfs.shopkey ,vfs.partkey,vfs.scskey,vfs.brand,vfs.season,vfs.partcode,vfs.color,vfs.siz,vfs.week_takeout_qty,vfs.takeout_qty ,vfs.week_sale_qty ,vfs.sale_qty,vfs.week_sale_amt,vfs.stock_qty ,vfs.sale_amt,prdt.gender 
FROM ax.vw_f_scsshopweek vfs
INNER JOIN ax.vw_d_prdt prdt ON vfs.scskey  = prdt.scskey
WHERE vfs.brand = 'X' AND vfs.weekkey BETWEEN '2018-01-01' AND '2020-12-31' AND prdt.item ='SH'
'''

cur.execute(total_shop_stock_ori)
total_shop_stock_ori = cur.fetchall()

print("time :", time.time() - start)  # 현재시각 - 시작시간 = 실행 시간

KeyboardInterrupt: 

In [ ]:
total_shop_stock_ori = pd.DataFrame(total_shop_stock_ori)
total_shop_stock_ori.columns = ['weekkey','shopkey','partkey','scskey','brand','season','partcode','color','siz','week_takeout_qty','takeout_qty' ,'week_sale_qty' ,'sale_qty','week_sale_amt','stock_qty' ,'sale_amt','gender']

In [422]:
total_shop_stock.to_csv("total_shop_stock.csv")

In [ ]:
total_shop_stock = total_shop_stock_ori

In [421]:
total_shop_stock

,weekkey,shopkey,partkey,scskey,brand,season,partcode,color,siz,week_takeout_qty,takeout_qty,week_sale_qty,sale_qty,week_sale_amt,stock_qty,sale_amt,gender
0,2018-12-10,X50055,X18FDXSH03841,X18FDXSH03841BK240,X,18F,DXSH03841,BK,240,3,6,1,3,134100.00,3,412100.00,X
1,2019-06-24,X50055,X18FDXSH03841,X18FDXSH03841BK240,X,18F,DXSH03841,BK,240,0,4,0,3,0.00,1,412100.00,X
2,2019-06-24,X50055,X18FDXSH03841,X18FDXSH03841GR265,X,18F,DXSH03841,GR,265,0,1,0,0,0.00,1,0.00,X
3,2018-12-10,X50055,X18FDXSH03841,X18FDXSH03841NY255,X,18F,DXSH03841,NY,255,1,1,0,0,0.00,1,0.00,X
4,2019-06-24,X50055,X18FDXSH03841,X18FDXSH03841NY255,X,18F,DXSH03841,NY,255,0,1,0,0,0.00,1,0.00,X
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2840150,2020-11-23,X50007,X20SDXSHA5011,X20SDXSHA5011OW250,X,20S,DXSHA5011,OW,250,0,6,1,4,139000.00,2,528050.00,X
2840151,2019-12-30,X50007,X20SDXSHA5011,X20SDXSHA5011OW280,X,20S,DXSHA5011,OW,280,1,2,0,1,0.00,1,132050.00,X
2840152,2020-03-23,X50007,X20SDXSHA5011,X20SDXSHA5011OW280,X,20S,DXSHA5011,OW,280,0,3,0,2,0.00,1,264100.00,X
2840153,2020-08-17,X50007,X20SDXSHB1031,X20SDXSHB1031OW250,X,20S,DXSHB1031,OW,250,-1,1,0,1,0.00,0,139000.00,X


In [ ]:
total_shop_refine_stock = total_shop_stock[ total_shop_stock['partkey'].isin(total_shop_stock_ori['partkey'])]
# 전체 날짜별 사이즈 확인
test_siz = total_shop_refine_stock.groupby(['weekkey','siz']).sum()
test_color = total_shop_refine_stock.groupby(['weekkey','color']).sum()

test_siz = test_siz.reset_index()
test_color = test_color.reset_index()


In [ ]:
test_siz['ratio'] =test_siz['sale_qty'] /test_siz['takeout_qty']
test_siz['out_sale'] =test_siz['week_takeout_qty'] - test_siz['week_sale_qty']

test_color['ratio'] =test_color['sale_qty'] /test_color['takeout_qty']
test_color['out_sale'] =test_color['week_takeout_qty'] - test_color['week_sale_qty']

In [ ]:
siz_list = sorted( test_siz['siz'].unique() )
siz_lists = list(map(int, siz_list))
siz_list = []
for i in siz_lists:
    if i >=230:
        siz_list.append(str(i))
siz_list

In [ ]:
# 위탁 입고량
plt.plot(test_siz[test_siz['siz']=='225']['weekkey'], test_siz[test_siz['siz']=='225']['takeout_qty'],label='225')
plt.plot(test_siz[test_siz['siz']=='230']['weekkey'], test_siz[test_siz['siz']=='230']['takeout_qty'],label='230')
plt.plot(test_siz[test_siz['siz']=='240']['weekkey'], test_siz[test_siz['siz']=='240']['takeout_qty'],label='240')
plt.plot(test_siz[test_siz['siz']=='250']['weekkey'], test_siz[test_siz['siz']=='250']['takeout_qty'],label='250')
plt.plot(test_siz[test_siz['siz']=='260']['weekkey'], test_siz[test_siz['siz']=='260']['takeout_qty'],label='260')
plt.plot(test_siz[test_siz['siz']=='270']['weekkey'], test_siz[test_siz['siz']=='270']['takeout_qty'],label='270')
plt.plot(test_siz[test_siz['siz']=='280']['weekkey'], test_siz[test_siz['siz']=='280']['takeout_qty'],label='280')
plt.plot(test_siz[test_siz['siz']=='290']['weekkey'], test_siz[test_siz['siz']=='290']['takeout_qty'],label='290')

plt.legend([225,230,240,250,260,270,280,290])
plt.xlabel('주')
plt.ylabel('입고량')
plt.title('총 위탁 입고량')
plt.show()

In [ ]:
# 위탁 판매량
plt.plot(test_siz[test_siz['siz']=='225']['weekkey'], test_siz[test_siz['siz']=='225']['sale_qty'],label='225')
plt.plot(test_siz[test_siz['siz']=='230']['weekkey'], test_siz[test_siz['siz']=='230']['sale_qty'],label='230')
plt.plot(test_siz[test_siz['siz']=='240']['weekkey'], test_siz[test_siz['siz']=='240']['sale_qty'],label='240')
plt.plot(test_siz[test_siz['siz']=='250']['weekkey'], test_siz[test_siz['siz']=='250']['sale_qty'],label='250')
plt.plot(test_siz[test_siz['siz']=='260']['weekkey'], test_siz[test_siz['siz']=='260']['sale_qty'],label='260')
plt.plot(test_siz[test_siz['siz']=='270']['weekkey'], test_siz[test_siz['siz']=='270']['sale_qty'],label='270')
plt.plot(test_siz[test_siz['siz']=='280']['weekkey'], test_siz[test_siz['siz']=='280']['sale_qty'],label='280')
plt.plot(test_siz[test_siz['siz']=='290']['weekkey'], test_siz[test_siz['siz']=='280']['sale_qty'],label='290')

plt.legend([225,230,240,250,260,270,280,290])
plt.xlabel('주')
plt.ylabel('판매량')
plt.title('총 위탁 판매량')
plt.show()

In [ ]:
# 매장내 재고 수 stock_qty
plt.plot(test_siz[test_siz['siz']=='225']['weekkey'], test_siz[test_siz['siz']=='225']['stock_qty'],label='225')
plt.plot(test_siz[test_siz['siz']=='230']['weekkey'], test_siz[test_siz['siz']=='230']['stock_qty'],label='230')
plt.plot(test_siz[test_siz['siz']=='240']['weekkey'], test_siz[test_siz['siz']=='240']['stock_qty'],label='240')
plt.plot(test_siz[test_siz['siz']=='250']['weekkey'], test_siz[test_siz['siz']=='250']['stock_qty'],label='250')
plt.plot(test_siz[test_siz['siz']=='260']['weekkey'], test_siz[test_siz['siz']=='260']['stock_qty'],label='260')
plt.plot(test_siz[test_siz['siz']=='270']['weekkey'], test_siz[test_siz['siz']=='270']['stock_qty'],label='270')
plt.plot(test_siz[test_siz['siz']=='280']['weekkey'], test_siz[test_siz['siz']=='280']['stock_qty'],label='280')
plt.plot(test_siz[test_siz['siz']=='290']['weekkey'], test_siz[test_siz['siz']=='280']['stock_qty'],label='290')
plt.legend([225,230,240,250,260,270,280,290])
plt.xlabel('주')
plt.ylabel('재고량')
plt.title('총 매장 내 재고량')
plt.show()

In [ ]:
test_total = test_siz.groupby(['weekkey']).sum()
test_total

In [ ]:
# 매장내 재고 수 out_sale = takeout_qty - sale_qty
plt.plot(test_total.index, test_total['stock_qty'],label='재고량')
plt.xlabel('주')
plt.ylabel('총 재고량(매장)')
plt.show()

In [ ]:
start = time.time()

total_stock = '''
SELECT mfs.weekkey,mfs.partkey,mfs.scskey,mfs.brand,mfs.season,mfs.partcode,mfs.color,mfs.siz,mfs.week_purch_qty,mfs.week_accept_qty,mfs.week_domestic_sale_qty,mfs.week_csm_sale_qty,mfs.purch_qty,mfs.accept_qty,mfs.takeout_qty,mfs.csm_takeout_qty,mfs.domestic_sale_qty,mfs.csm_sale_qty,mfs.stock_qty,mfs.wms_stock_qty,prdt.gender 
FROM ax.vw_f_scsweek mfs
INNER JOIN ax.vw_d_prdt prdt ON mfs.scskey  = prdt.scskey
WHERE mfs.brand = 'X' AND mfs.weekkey BETWEEN '2018-01-01' AND '2021-03-01' AND prdt.item ='SH'
'''

cur.execute(total_stock)
total_stock = cur.fetchall()

print("time :", time.time() - start)  # 현재시각 - 시작시간 = 실행 시간

In [ ]:
total_stock = pd.DataFrame(total_stock)
total_stock.columns = ['weekkey','partkey','scskey','brand','season','partcode','color','siz','week_purch_qty','week_accept_qty','week_domestic_sale_qty','week_csm_sale_qty','purch_qty','accept_qty','takeout_qty','csm_takeout_qty','domestic_sale_qty','csm_sale_qty','stock_qty','wms_stock_qty','gender']

In [ ]:
test1 = total_stock.groupby(['weekkey','siz']).sum()
test2 = test1.reset_index()


In [ ]:
test2['ratio'] = test2['csm_sale_qty']/ test2['csm_takeout_qty'] 
test2['store_inventory'] =test2['csm_takeout_qty'] - test2['csm_sale_qty']

# test_color['ratio'] =test_color['csm_takeout_qty'] /test_color['takeout_qty']
# test_color['out_sale'] =test_color['week_takeout_qty'] - test_color['week_sale_qty']

In [ ]:
siz_list = sorted(test2['siz'].unique())

In [ ]:
test2

In [ ]:
# 위탁 출고량
plt.plot(test2[test2['siz']=='230']['weekkey'], test2[test2['siz']=='230']['csm_takeout_qty'],label='230')
plt.plot(test2[test2['siz']=='240']['weekkey'], test2[test2['siz']=='240']['csm_takeout_qty'],label='240')
plt.plot(test2[test2['siz']=='250']['weekkey'], test2[test2['siz']=='250']['csm_takeout_qty'],label='250')
plt.plot(test2[test2['siz']=='260']['weekkey'], test2[test2['siz']=='260']['csm_takeout_qty'],label='260')
plt.plot(test2[test2['siz']=='270']['weekkey'], test2[test2['siz']=='270']['csm_takeout_qty'],label='270')
plt.plot(test2[test2['siz']=='280']['weekkey'], test2[test2['siz']=='280']['csm_takeout_qty'],label='280')
plt.legend([230,240,250,260,270,280])
plt.xlabel('주')
plt.ylabel('위탁 출고량')
plt.show()

In [ ]:
# 위탁 출고량
plt.plot(test2[test2['siz']=='230']['weekkey'], test2[test2['siz']=='230']['csm_sale_qty'],label='230')
plt.plot(test2[test2['siz']=='240']['weekkey'], test2[test2['siz']=='240']['csm_sale_qty'],label='240')
plt.plot(test2[test2['siz']=='250']['weekkey'], test2[test2['siz']=='250']['csm_sale_qty'],label='250')
plt.plot(test2[test2['siz']=='260']['weekkey'], test2[test2['siz']=='260']['csm_sale_qty'],label='260')
plt.plot(test2[test2['siz']=='270']['weekkey'], test2[test2['siz']=='270']['csm_sale_qty'],label='270')
plt.plot(test2[test2['siz']=='280']['weekkey'], test2[test2['siz']=='280']['csm_sale_qty'],label='280')
plt.legend([230,240,250,260,270,280])
plt.xlabel('주')
plt.ylabel('위탁 출고량')
plt.show()

In [ ]:
# 위탁 출고량
plt.plot(test2[test2['siz']=='230']['weekkey'], test2[test2['siz']=='230']['store_inventory'],label='230')
plt.plot(test2[test2['siz']=='240']['weekkey'], test2[test2['siz']=='240']['store_inventory'],label='240')
plt.plot(test2[test2['siz']=='250']['weekkey'], test2[test2['siz']=='250']['store_inventory'],label='250')
plt.plot(test2[test2['siz']=='260']['weekkey'], test2[test2['siz']=='260']['store_inventory'],label='260')
plt.plot(test2[test2['siz']=='270']['weekkey'], test2[test2['siz']=='270']['store_inventory'],label='270')
plt.plot(test2[test2['siz']=='280']['weekkey'], test2[test2['siz']=='280']['store_inventory'],label='280')
plt.legend([230,240,250,260,270,280])
plt.xlabel('주')
plt.ylabel('매장 재고량')
plt.show()

In [ ]:
# 위탁 출고량
plt.plot(test2[test2['siz']=='230']['weekkey'], test2[test2['siz']=='230']['ratio'],label='230')
plt.plot(test2[test2['siz']=='240']['weekkey'], test2[test2['siz']=='240']['ratio'],label='240')
plt.plot(test2[test2['siz']=='250']['weekkey'], test2[test2['siz']=='250']['ratio'],label='250')
plt.plot(test2[test2['siz']=='260']['weekkey'], test2[test2['siz']=='260']['ratio'],label='260')
plt.plot(test2[test2['siz']=='270']['weekkey'], test2[test2['siz']=='270']['ratio'],label='270')
plt.plot(test2[test2['siz']=='280']['weekkey'], test2[test2['siz']=='280']['ratio'],label='280')
plt.legend([230,240,250,260,270,280])
plt.xlabel('주')
plt.ylabel('매장 재고량')
plt.show()

In [361]:
start = time.time()

shop_list = '''
SELECT shop_2019.shopkey 
FROM(SELECT *
FROM ax.vw_d_shop vds
WHERE type_ax_zone_nm ='국내' AND  vds.chnl_type_sfdc ='오프라인' AND shop_nm NOT LIKE '%아울렛%' AND shop_type_sfdc NOT LIKE '%상설%'  AND vds.brand = 'X' AND (closedate > '2019-12-31' OR closedate is NULL ) AND opendate < '2019-01-01'
LIMIT 1000) AS shop_2019
INNER JOIN (SELECT *
FROM ax.vw_d_shop vds
WHERE type_ax_zone_nm ='국내' AND  vds.chnl_type_sfdc ='오프라인' AND shop_nm NOT LIKE '%아울렛%' AND shop_type_sfdc NOT LIKE '%상설%' AND vds.brand = 'X' AND (closedate > '2020-12-31' OR closedate is NULL ) AND opendate < '2020-01-01'
) AS shop_2020
ON shop_2019.shopkey =shop_2020.shopkey
'''

cur.execute(shop_list)
shop_list = cur.fetchall()

print("time :", time.time() - start)  # 현재시각 - 시작시간 = 실행 시간

time : 1.289707899093628


In [362]:
shop_list_pd = pd.DataFrame(shop_list)
shop_list_pd.columns = ['shopkey']

In [418]:
shop_list_pd.to_csv("shop_list.csv")

In [363]:
len(shop_list_pd['shopkey'].unique())

143

In [364]:
shop_list_pd[shop_list_pd['shopkey']=='X50002']

,shopkey
68,X50002


In [365]:
total_shop_stock

,weekkey,shopkey,partkey,scskey,brand,season,partcode,color,siz,week_takeout_qty,takeout_qty,week_sale_qty,sale_qty,week_sale_amt,stock_qty,sale_amt,gender
0,2018-12-10,X50055,X18FDXSH03841,X18FDXSH03841BK240,X,18F,DXSH03841,BK,240,3,6,1,3,134100.00,3,412100.00,X
1,2019-06-24,X50055,X18FDXSH03841,X18FDXSH03841BK240,X,18F,DXSH03841,BK,240,0,4,0,3,0.00,1,412100.00,X
2,2019-06-24,X50055,X18FDXSH03841,X18FDXSH03841GR265,X,18F,DXSH03841,GR,265,0,1,0,0,0.00,1,0.00,X
3,2018-12-10,X50055,X18FDXSH03841,X18FDXSH03841NY255,X,18F,DXSH03841,NY,255,1,1,0,0,0.00,1,0.00,X
4,2019-06-24,X50055,X18FDXSH03841,X18FDXSH03841NY255,X,18F,DXSH03841,NY,255,0,1,0,0,0.00,1,0.00,X
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2840150,2020-11-23,X50007,X20SDXSHA5011,X20SDXSHA5011OW250,X,20S,DXSHA5011,OW,250,0,6,1,4,139000.00,2,528050.00,X
2840151,2019-12-30,X50007,X20SDXSHA5011,X20SDXSHA5011OW280,X,20S,DXSHA5011,OW,280,1,2,0,1,0.00,1,132050.00,X
2840152,2020-03-23,X50007,X20SDXSHA5011,X20SDXSHA5011OW280,X,20S,DXSHA5011,OW,280,0,3,0,2,0.00,1,264100.00,X
2840153,2020-08-17,X50007,X20SDXSHB1031,X20SDXSHB1031OW250,X,20S,DXSHB1031,OW,250,-1,1,0,1,0.00,0,139000.00,X


In [366]:
total_shop_stock1920 = total_shop_stock[total_shop_stock['weekkey']> datetime.date(2018,12,31) ]

In [367]:
total_shop_stock1920[total_shop_stock['weekkey']> datetime.date(2018,12,31) ]

<ipython-input-367-bac50175085a>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  total_shop_stock1920[total_shop_stock['weekkey']> datetime.date(2018,12,31) ]


,weekkey,shopkey,partkey,scskey,brand,season,partcode,color,siz,week_takeout_qty,takeout_qty,week_sale_qty,sale_qty,week_sale_amt,stock_qty,sale_amt,gender
1,2019-06-24,X50055,X18FDXSH03841,X18FDXSH03841BK240,X,18F,DXSH03841,BK,240,0,4,0,3,0.00,1,412100.00,X
2,2019-06-24,X50055,X18FDXSH03841,X18FDXSH03841GR265,X,18F,DXSH03841,GR,265,0,1,0,0,0.00,1,0.00,X
4,2019-06-24,X50055,X18FDXSH03841,X18FDXSH03841NY255,X,18F,DXSH03841,NY,255,0,1,0,0,0.00,1,0.00,X
5,2019-06-24,X50055,X18FDXSH03841,X18FDXSH03841NY260,X,18F,DXSH03841,NY,260,0,1,0,0,0.00,1,0.00,X
6,2019-06-24,X50055,X18FDXSH03841,X18FDXSH03841NY265,X,18F,DXSH03841,NY,265,0,2,0,1,0.00,1,134100.00,X
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2840150,2020-11-23,X50007,X20SDXSHA5011,X20SDXSHA5011OW250,X,20S,DXSHA5011,OW,250,0,6,1,4,139000.00,2,528050.00,X
2840151,2019-12-30,X50007,X20SDXSHA5011,X20SDXSHA5011OW280,X,20S,DXSHA5011,OW,280,1,2,0,1,0.00,1,132050.00,X
2840152,2020-03-23,X50007,X20SDXSHA5011,X20SDXSHA5011OW280,X,20S,DXSHA5011,OW,280,0,3,0,2,0.00,1,264100.00,X
2840153,2020-08-17,X50007,X20SDXSHB1031,X20SDXSHB1031OW250,X,20S,DXSHB1031,OW,250,-1,1,0,1,0.00,0,139000.00,X


In [368]:
df_INNER_JOIN = pd.merge(shop_list_pd, total_shop_stock1920, left_on='shopkey', right_on='shopkey', how='inner')

In [369]:
len(df_INNER_JOIN['shopkey'].unique())

143

In [370]:
shop_stock_2019 = df_INNER_JOIN[df_INNER_JOIN['weekkey']< datetime.date(2020,1,1) ]

In [371]:
shop_stock_2020 = df_INNER_JOIN[df_INNER_JOIN['weekkey']> datetime.date(2019,12,31) ]

In [372]:
shop_stock_2020

,shopkey,weekkey,partkey,scskey,brand,season,partcode,color,siz,week_takeout_qty,takeout_qty,week_sale_qty,sale_qty,week_sale_amt,stock_qty,sale_amt,gender
48,X10052,2020-04-13,X18FDXSH32841,X18FDXSH32841BK280,X,18F,DXSH32841,BK,280,1,2,1,2,54500.00,0,109000.00,X
58,X10052,2020-03-02,X18FDXSH43841,X18FDXSH43841GN280,X,18F,DXSH43841,GN,280,2,5,2,5,109000.00,0,272500.00,X
59,X10052,2020-04-13,X18FDXSH43841,X18FDXSH43841GN280,X,18F,DXSH43841,GN,280,0,5,-1,4,-54500.00,1,218000.00,X
60,X10052,2020-03-23,X18FDXSH44841,X18FDXSH44841RD240,X,18F,DXSH44841,RD,240,1,2,1,2,54500.00,0,109000.00,X
61,X10052,2020-04-13,X18FDXSH44841,X18FDXSH44841RD240,X,18F,DXSH44841,RD,240,2,4,2,4,109000.00,0,218000.00,X
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1473362,X50096,2020-12-28,X21SDXSHJ1111,X21SDXSHJ1111BG270,X,21S,DXSHJ1111,BG,270,1,1,0,0,0.00,1,0.00,X
1473363,X50096,2020-12-28,X21SDXSHJ1111,X21SDXSHJ1111BK245,X,21S,DXSHJ1111,BK,245,1,1,0,0,0.00,1,0.00,X
1473364,X50096,2020-12-28,X21SDXSHJ1111,X21SDXSHJ1111BK270,X,21S,DXSHJ1111,BK,270,1,1,0,0,0.00,1,0.00,X
1473365,X50096,2020-12-28,X21SDXSHJ1111,X21SDXSHJ1111WH260,X,21S,DXSHJ1111,WH,260,1,1,0,0,0.00,1,0.00,X


In [373]:
# 전체 날짜별 사이즈 확인
stock_2019 = shop_stock_2019.groupby(['weekkey','siz']).sum()
stock_2020 = shop_stock_2020.groupby(['weekkey','color']).sum()

stock_2019 = stock_2019.reset_index()
stock_2020 = stock_2020.reset_index()

# test_color = test_color.reset_index()

In [374]:
shop_stock_2019['week_sale_qty'].sum()

331493

In [375]:
stock_2020['week_sale_qty'].sum()

400539

In [376]:
shop_stock_2019['week_sale_amt'].sum()

Decimal('29500592880.00')

In [377]:
shop_stock_2020['week_sale_amt'].sum()

Decimal('38078196776.00')

In [378]:
shop_stock_2020['week_sale_qty'].sum() / shop_stock_2019['week_sale_qty'].sum()* 100

120.82879578150971

In [379]:
shop_stock_2020['week_sale_amt'].sum() / shop_stock_2019['week_sale_amt'].sum()* 100

Decimal('129.0760390168809380213378274')

In [380]:
stock_2019 = shop_stock_2019.groupby(['weekkey','shopkey',]).sum()
stock_2020 = shop_stock_2020.groupby(['weekkey','shopkey']).sum()

stock_2019 = stock_2019.reset_index()
stock_2020 = stock_2020.reset_index()

In [381]:
stock_2019

,weekkey,shopkey,week_takeout_qty,takeout_qty,week_sale_qty,sale_qty,stock_qty
0,2019-01-07,X10003,0,2,0,1,1
1,2019-01-07,X10004,73,325,36,259,66
2,2019-01-07,X10005,2,22,2,16,6
3,2019-01-07,X10006,31,105,30,54,51
4,2019-01-07,X10007,-1,15,2,11,4
...,...,...,...,...,...,...,...
7421,2019-12-30,X50128,82,293,34,173,120
7422,2019-12-30,X50129,121,367,64,196,171
7423,2019-12-30,X50130,163,633,76,389,244
7424,2019-12-30,X50131,122,359,29,172,187


In [382]:
shop_key_list = stock_2019['shopkey'].unique()

In [383]:
stock_sale_2020

[3816,
 4013,
 2429,
 8136,
 6249,
 5640,
 4432,
 3326,
 3619,
 2220,
 2541,
 6212,
 2372,
 2029,
 4231,
 5617,
 4000,
 2844,
 1747,
 5178,
 2051,
 1174,
 1012,
 4350,
 5158,
 2594,
 4451,
 1323,
 3100,
 1951,
 2653,
 4197,
 3511,
 1263,
 2347,
 1508,
 2612,
 11534,
 3878,
 2014,
 4308,
 6533,
 2702,
 5795,
 3017,
 2735,
 7284,
 6506,
 1587,
 2497,
 6923,
 1783,
 3828,
 2407,
 5049,
 7052,
 2301,
 5065,
 2709,
 4571,
 3789,
 4829,
 2295,
 2201,
 2929,
 1747,
 1309,
 4765,
 1276,
 2220,
 1895,
 3984,
 1911,
 3066,
 2391,
 1690,
 1242,
 1117,
 1732,
 2539,
 1904,
 2357,
 1433,
 2245,
 2338,
 1199,
 2608,
 1548,
 1318,
 2423,
 1792,
 2616,
 1170,
 1434,
 2950,
 1144,
 1406,
 4467,
 1816,
 3043,
 1408,
 1880,
 2308,
 1539,
 1127,
 1171,
 1085,
 3050,
 1675,
 1023,
 1625,
 1508,
 1289,
 3361,
 1912,
 1329,
 2753,
 1847,
 1485,
 1657,
 2249,
 1455,
 1332,
 1473,
 1644,
 1855,
 1573,
 2428,
 2226,
 2431,
 1691,
 2012,
 2097,
 1419,
 2861,
 2219,
 3271,
 2755,
 1256,
 3212,
 2483,
 1443,
 2000

In [295]:
stock_2019[stock_2019['shopkey']==i][stock_2019['stock_qty']==0].count()

<ipython-input-295-20eb9d8e3218>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  stock_2019[stock_2019['shopkey']==i][stock_2019['stock_qty']==0].count()


weekkey             0
shopkey             0
week_takeout_qty    0
takeout_qty         0
week_sale_qty       0
sale_qty            0
stock_qty           0
dtype: int64

In [385]:
stock_sum_2019 = []
stock_sum_2020 = []
stock_plus_2019 = []
stock_takeout_2019 = []
stock_takeout_2020 = []
stock_sale_2019 = []
stock_sale_2020 = []
stock_ratio_2019=[]
stock_ratio_2020=[]

for i in shop_key_list:
    stock_sale_2019.append(stock_2019[stock_2019['shopkey']==i]['week_sale_qty'].sum())
    stock_plus_2019.append( round(stock_2019[stock_2019['shopkey']==i]['week_sale_qty'].sum()*1.2))
    stock_sale_2020.append(stock_2020[stock_2020['shopkey']==i]['week_sale_qty'].sum())
    stock_takeout_2019.append(stock_2019[stock_2019['shopkey']==i]['week_takeout_qty'].sum())
    stock_takeout_2020.append(stock_2020[stock_2020['shopkey']==i]['week_takeout_qty'].sum())
#     stock_sale_2019.append(stock_2019[stock_2019['shopkey']==i]['sale_qty'].sum())
#     stock_sale_2020.append(stock_2020[stock_2020['shopkey']==i]['sale_qty'].sum())
    stock_ratio_2019.append(stock_2019[stock_2019['shopkey']==i].sum()['stock_qty'] / stock_2019[stock_2019['shopkey']==i].sum()['takeout_qty'])
    stock_ratio_2020.append(stock_2020[stock_2020['shopkey']==i].sum()['stock_qty'] / stock_2020[stock_2020['shopkey']==i].sum()['takeout_qty'])

#     stock_ratio_2019.append( stock_2019[stock_2019['shopkey']==i]['week_sale_qty'].sum()/ )
#     stock_ratio_2020.append()
#     break
    

In [409]:
df = pd.DataFrame(
                {'shop_key_list' : shop_key_list,
                 '2019_주 별 총 판매량' : stock_sale_2019,
                '2019_주 별 총 입고량':stock_takeout_2019,
                 'stock_plus_2019' : stock_plus_2019,
                 '2020_주 별 총 판매량' : stock_sale_2020,
                 '2020_주 별 총 입고량':stock_takeout_2020,
#                  'stock_takeout_2020':stock_takeout_2020,
#                  'stock_sale_2020':stock_sale_2020,
                 '2019_재고비율':stock_ratio_2019,
                 '2020_재고비율':stock_ratio_2020
                },
                )

df

,shop_key_list,2019_주 별 총 판매량,2019_주 별 총 입고량,stock_plus_2019,2020_주 별 총 판매량,2020_주 별 총 입고량,2019_재고비율,2020_재고비율
0,X10003,2312,2590,2774,3816,4162,0.280029,0.176666
1,X10004,4745,5149,5694,4013,4109,0.212972,0.202442
2,X10005,2123,2514,2548,2429,2525,0.312090,0.212287
3,X10006,4603,5232,5524,8136,8130,0.232188,0.141909
4,X10007,4426,4864,5311,6249,6353,0.224536,0.151126
5,X10008,8846,9540,10615,5640,5574,0.135116,0.101314
6,X10009,5645,6395,6774,4432,4399,0.176070,0.186418
7,X10011,3986,4497,4783,3326,3417,0.254708,0.190916
8,X10014,2882,3456,3458,3619,3604,0.241424,0.228667
9,X10015,1554,1724,1865,2220,2556,0.350087,0.245579


In [410]:
df['19년 대비 20년 판매 비율'] = df['2020_주 별 총 판매량'] / df['2019_주 별 총 판매량']

In [411]:
df['19년대비 20년 입고 비율'] = df['2020_주 별 총 입고량'] / df['2019_주 별 총 입고량']

In [412]:
df

,shop_key_list,2019_주 별 총 판매량,2019_주 별 총 입고량,stock_plus_2019,2020_주 별 총 판매량,2020_주 별 총 입고량,2019_재고비율,2020_재고비율,19년 대비 20년 판매 비율,19년대비 20년 입고 비율
0,X10003,2312,2590,2774,3816,4162,0.280029,0.176666,1.650519,1.606950
1,X10004,4745,5149,5694,4013,4109,0.212972,0.202442,0.845732,0.798019
2,X10005,2123,2514,2548,2429,2525,0.312090,0.212287,1.144136,1.004375
3,X10006,4603,5232,5524,8136,8130,0.232188,0.141909,1.767543,1.553899
4,X10007,4426,4864,5311,6249,6353,0.224536,0.151126,1.411884,1.306127
5,X10008,8846,9540,10615,5640,5574,0.135116,0.101314,0.637576,0.584277
6,X10009,5645,6395,6774,4432,4399,0.176070,0.186418,0.785120,0.687881
7,X10011,3986,4497,4783,3326,3417,0.254708,0.190916,0.834420,0.759840
8,X10014,2882,3456,3458,3619,3604,0.241424,0.228667,1.255725,1.042824
9,X10015,1554,1724,1865,2220,2556,0.350087,0.245579,1.428571,1.482599


In [413]:
df['입고 대비 판매량'] = df['19년 대비 20년 판매 비율'] / df['19년대비 20년 입고 비율']

In [414]:
df.sort_values(by=['입고 대비 판매량'],ascending=False)

,shop_key_list,2019_주 별 총 판매량,2019_주 별 총 입고량,stock_plus_2019,2020_주 별 총 판매량,2020_주 별 총 입고량,2019_재고비율,2020_재고비율,19년 대비 20년 판매 비율,19년대비 20년 입고 비율,입고 대비 판매량
93,X50057,745,1076,894,1434,1476,0.471829,0.315164,1.924832,1.371747,1.403197
43,X10057,2769,3571,3323,5795,5584,0.327739,0.196783,2.092813,1.563708,1.338366
117,X50102,2600,3002,3120,1847,1625,0.445128,0.266046,0.710385,0.541306,1.312354
19,X10027,2909,3720,3491,5178,5150,0.311182,0.294999,1.779993,1.384409,1.285743
129,X50118,1729,2169,2075,2431,2414,0.414204,0.289489,1.406015,1.112955,1.263317
51,X10065,2032,2485,2438,1783,1768,0.287342,0.294306,0.877461,0.711469,1.233309
21,X10029,1827,2221,2192,1174,1171,0.310062,0.274731,0.642583,0.527240,1.218768
8,X10014,2882,3456,3458,3619,3604,0.241424,0.228667,1.255725,1.042824,1.204158
75,X50005,965,1232,1158,1690,1800,0.412378,0.296976,1.751295,1.461039,1.198664
95,X50059,1451,1692,1741,1144,1115,0.409060,0.335875,0.788422,0.658983,1.196421


In [415]:
refine_df = df[ (df['19년 대비 20년 판매 비율']>1.0) & (df['입고 대비 판매량']>1.0) ].sort_values(by=['19년 대비 20년 판매 비율'],ascending=False)
refine_df

,shop_key_list,2019_주 별 총 판매량,2019_주 별 총 입고량,stock_plus_2019,2020_주 별 총 판매량,2020_주 별 총 입고량,2019_재고비율,2020_재고비율,19년 대비 20년 판매 비율,19년대비 20년 입고 비율,입고 대비 판매량
55,X10070,2797,3201,3356,7052,7344,0.269065,0.145883,2.521273,2.294283,1.098937
43,X10057,2769,3571,3323,5795,5584,0.327739,0.196783,2.092813,1.563708,1.338366
50,X10064,3424,3920,4109,6923,7318,0.266872,0.172663,2.021904,1.866837,1.083064
37,X10050,5722,6405,6866,11534,12274,0.287398,0.166461,2.015729,1.916315,1.051877
93,X50057,745,1076,894,1434,1476,0.471829,0.315164,1.924832,1.371747,1.403197
36,X10048,1365,1577,1638,2612,2832,0.336340,0.225944,1.913553,1.795815,1.065563
42,X10056,1461,1647,1753,2702,2840,0.356620,0.217630,1.849418,1.724347,1.072532
92,X50055,657,816,788,1170,1285,0.474767,0.338614,1.780822,1.574755,1.130857
19,X10027,2909,3720,3491,5178,5150,0.311182,0.294999,1.779993,1.384409,1.285743
3,X10006,4603,5232,5524,8136,8130,0.232188,0.141909,1.767543,1.553899,1.137489


In [416]:
len(refine_df)

82

In [417]:
refine_df.corr()

,2019_주 별 총 판매량,2019_주 별 총 입고량,stock_plus_2019,2020_주 별 총 판매량,2020_주 별 총 입고량,2019_재고비율,2020_재고비율,19년 대비 20년 판매 비율,19년대비 20년 입고 비율,입고 대비 판매량
2019_주 별 총 판매량,1.000000,0.995387,1.000000,0.878489,0.874276,-0.860918,-0.821762,-0.094565,-0.072593,-0.079771
2019_주 별 총 입고량,0.995387,1.000000,0.995393,0.887132,0.881445,-0.863031,-0.818925,-0.062488,-0.066617,-0.000312
stock_plus_2019,1.000000,0.995393,1.000000,0.878480,0.874266,-0.860922,-0.821751,-0.094590,-0.072634,-0.079727
2020_주 별 총 판매량,0.878489,0.887132,0.878480,1.000000,0.998399,-0.715884,-0.770256,0.345518,0.354137,0.013545
2020_주 별 총 입고량,0.874276,0.881445,0.874266,0.998399,1.000000,-0.714291,-0.770265,0.349804,0.369406,-0.015726
2019_재고비율,-0.860918,-0.863031,-0.860922,-0.715884,-0.714291,1.000000,0.884104,0.119404,0.093404,0.089166
2020_재고비율,-0.821762,-0.818925,-0.821751,-0.770256,-0.770265,0.884104,1.000000,-0.087589,-0.118995,0.082073
19년 대비 20년 판매 비율,-0.094565,-0.062488,-0.094590,0.345518,0.349804,0.119404,-0.087589,1.000000,0.941569,0.281972
19년대비 20년 입고 비율,-0.072593,-0.066617,-0.072634,0.354137,0.369406,0.093404,-0.118995,0.941569,1.000000,-0.054203
입고 대비 판매량,-0.079771,-0.000312,-0.079727,0.013545,-0.015726,0.089166,0.082073,0.281972,-0.054203,1.000000


In [287]:
# 판매비율 보다 더 늘어난 매장분석

df[df['stock_sum_2020']>df['stock_plus_2019']] 

KeyError: 'stock_sum_2020'

In [ ]:
df['sale2019_stock2019'] = df['stock_sale_2019'] / df['stock_takeout_2019']

In [ ]:
df['sale2020_stock2020'] = df['stock_sale_2020'] / df['stock_takeout_2020']

In [ ]:
df[df['sum_2020_sum_2019']>1.5]

In [ ]:
df[df['sum_2020_sum_2019']<1.0]

In [ ]:
df[df['sum_2020_sum_2019']<1.5]